# **Dataton - Data Ninjas**

In [1]:
pip install openpyxl

In [2]:
pip install geopandas shapely pandas

In [3]:
pip install pyodbc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.0/336.0 kB 4.7 MB/s eta 0:00:00


In [4]:
pip install matplotlib seaborn pandas

In [5]:
pip install folium

In [6]:
import pandas as pd
import re
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import imageio
import matplotlib.cm as cm
from scipy.stats import norm
from scipy import stats
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import roc_auc_score,recall_score,precision_score,f1_score,confusion_matrix,accuracy_score,classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import AdaBoostClassifier,BaggingClassifier,ExtraTreesClassifier,GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
import itertools
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans

**PROBLEMA**

¿Dar una compensación/ refund compensa la mala experiencia de un usuario? ¿La frecuencia y ticket de compra aumenta, disminuye o se mantiene?

**GRUPO A**

Usuarios con compensación o refund en jul-24 con categoría de usuario y su comportamiento antes y después

In [7]:
# my_project='peya-salvador'
# query=f'''
# SELECT *
# FROM peya-salvador.automated_tables_reports.Users_Compensados
# '''
# from google.cloud import bigquery
# df = bigquery.Client(project=my_project).query(query).result().to_dataframe()

# for column in df.columns:
#     if 'dbdate' in str(df[column].dtype):
#         df[column] = pd.to_datetime(df[column])

# for column in df.columns:
#     if pd.api.types.is_integer_dtype(df[column]):
#         df[column] = df[column].astype('float64')

In [8]:
df = pd.read_excel('BaseA.xlsx')

In [9]:
users_compensados = df[(df['afv_aft'].notna()) & (df['afv_aft'] != '') & (df['id_user_cluster'].notna()) & (df['id_user_cluster'] != '')]

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
users_compensados.info(10)
# 19756 usuarios

<class 'pandas.core.frame.DataFrame'>
Index: 19756 entries, 846 to 25517
Data columns (total 60 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   user_id_order        19756 non-null  int64         
 1   original_order_id    19756 non-null  int64         
 2   order_type           19756 non-null  object        
 3   refund_amount        19756 non-null  float64       
 4   compensation_amount  19756 non-null  float64       
 5   fecha                19756 non-null  datetime64[ns]
 6   afv_aft              19756 non-null  float64       
 7   gpo_aft              19756 non-null  float64       
 8   SF_Aft               19756 non-null  float64       
 9   DF_Aft               19756 non-null  float64       
 10  gp_fl_aft            19756 non-null  float64       
 11  take_in_aft          19756 non-null  float64       
 12  order_aft            19756 non-null  float64       
 13  ticket_bef           19443 non-nul

In [12]:
resumen_compensados = users_compensados.groupby('categoria').agg(
    users_tratamiento=('user_id_order', 'nunique'),
    orders_tratamiento=('original_order_id', 'nunique'),
    monto_comp= ('compensation_amount','mean'),
    freq_antes_tratamiento=('orders_bef', 'mean'),
    ticket_antes_tratamiento=('ticket_bef', 'mean'),
    freq_despues_tratamiento=('order_aft', 'mean'),
    ticket_despues_tratamiento=('afv_aft', 'mean'),
    gpo_despues_tratamiento=('gpo_aft', 'mean'),
    gpo_antes_tratamiento=('gpo_bef', 'mean'),
).reset_index()

resumen_compensados['var freq tratamiento'] = resumen_compensados['freq_despues_tratamiento']/resumen_compensados['freq_antes_tratamiento'] - 1
resumen_compensados['var ticket tratamiento'] = resumen_compensados['ticket_despues_tratamiento']/resumen_compensados['ticket_antes_tratamiento'] - 1

In [13]:
resumen_compensados

,categoria,users_tratamiento,orders_tratamiento,monto_comp,freq_antes_tratamiento,ticket_antes_tratamiento,freq_despues_tratamiento,ticket_despues_tratamiento,gpo_despues_tratamiento,gpo_antes_tratamiento,var freq tratamiento,var ticket tratamiento
0,About to Sleep,281,292,2.272641,2.029106,11.248487,1.472603,10.889550,0.916541,0.660667,-0.274260,-0.031910
1,At Risk,94,95,4.111111,2.468692,12.343596,1.684211,14.027309,1.443740,0.885034,-0.317772,0.136404
2,Can't Lose Them,22,24,4.483025,4.830251,14.916672,2.500000,16.986491,2.655750,1.311055,-0.482429,0.138759
3,Champion,8720,12913,1.827088,9.696023,12.058451,10.906606,12.360008,1.194576,0.801225,0.124854,0.025008
4,Hibernating,44,44,4.635943,1.550913,11.244524,1.590909,11.141307,0.803392,0.677519,0.025788,-0.009179
5,Loyal Customers,4447,5396,2.065570,5.673690,12.402354,4.307635,12.790113,1.296745,0.862285,-0.240770,0.031265
6,New Customer,69,69,1.983360,1.910150,10.546490,1.797101,9.563541,0.314094,0.633547,-0.059183,-0.093202
7,Potential Loyalists,877,923,1.933751,2.230637,10.812654,2.204767,11.082034,0.832176,0.564571,-0.011597,0.024913


**GRUPO B**

Usuarios con una mala experiencia de órdenes compensables y no se les atribuyó una compensación o refund en jul-24 con categoría de usuario y su comportamiento antes y después

In [14]:
# my_project='peya-salvador'
# query=f'''
# SELECT *
# FROM peya-salvador.automated_tables_reports.Users_Compensables
# '''
# from google.cloud import bigquery
# df = bigquery.Client(project=my_project).query(query).result().to_dataframe()

# for column in df.columns:
#     if 'dbdate' in str(df[column].dtype):
#         df[column] = pd.to_datetime(df[column])

# for column in df.columns:
#     if pd.api.types.is_integer_dtype(df[column]):
#         df[column] = df[column].astype('float64')

In [15]:
df2 = pd.read_excel('BaseB.xlsx')

In [16]:
users_compensables = df2
# 4563  usuarios
# > 20% de grupo control

In [17]:
users_compensables.info(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4563 entries, 0 to 4562
Data columns (total 57 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   user_id_order        4563 non-null   int64         
 1   fecha                4563 non-null   datetime64[ns]
 2   order_id             4563 non-null   int64         
 3   id_user              4563 non-null   int64         
 4   categoria            4563 non-null   object        
 5   user_id              4563 non-null   int64         
 6   gender               3815 non-null   object        
 7   income               4563 non-null   object        
 8   cell_segment         4563 non-null   object        
 9   cc_level             3804 non-null   object        
 10  segment              4563 non-null   object        
 11  age                  3549 non-null   float64       
 12  orders               4563 non-null   float64       
 13  cantidad_comercios   4563 non-nul

In [18]:
resumen_compensables = users_compensables.groupby('categoria').agg(
    users_control=('user_id_order', 'nunique'),
    orders_control=('order_id', 'nunique'),
    freq_antes_control=('orders_bef', 'mean'),
    ticket_antes_control=('ticket_bef', 'mean'),
    freq_despues_control=('order_aft', 'mean'),
    ticket_despues_control=('afv_aft', 'mean'),
        gpo_despues_control=('gpo_aft', 'mean'),
    gpo_antes_control=('gpo_bef', 'mean'),
).reset_index()

resumen_compensables['var freq control'] = resumen_compensables['freq_despues_control']/resumen_compensables['freq_antes_control'] - 1
resumen_compensables['var ticket control'] = resumen_compensables['ticket_despues_control']/resumen_compensables['ticket_antes_control'] - 1

In [19]:
resumen_compensables

,categoria,users_control,orders_control,freq_antes_control,ticket_antes_control,freq_despues_control,ticket_despues_control,gpo_despues_control,gpo_antes_control,var freq control,var ticket control
0,About to Sleep,164,173,2.439380,10.466871,1.653179,11.191927,0.893308,0.456048,-0.322295,0.069272
1,At Risk,90,97,2.854901,11.960070,1.783505,12.070683,1.053463,0.809856,-0.375283,0.009249
2,Can't Lose Them,14,14,6.095851,15.567411,2.357143,14.664381,1.431679,1.239343,-0.613320,-0.058008
3,Champion,2195,2454,8.348234,11.652561,8.087205,11.888607,1.052983,0.681288,-0.031268,0.020257
4,Hibernating,79,80,1.972603,9.888648,1.550000,10.641531,0.601516,0.412851,-0.214236,0.076136
5,Loyal Customers,1283,1365,4.970413,11.971607,3.515751,12.561719,1.169313,0.743866,-0.292664,0.049293
6,New Customer,27,28,1.703731,11.032385,1.464286,9.041137,0.453083,0.729267,-0.140541,-0.180491
7,Potential Loyalists,338,352,2.479740,10.170589,2.142045,10.710722,0.834286,0.287474,-0.136181,0.053107


## **Vamos a realizar un analisis de diferencias en diferencias para cada grupo de clientes**

In [20]:
# Combinar bases con la columna de categoría
users_compensables['group'] = 'control'
users_compensados['group'] = 'treatment'

# Combinar ambas bases de datos
data = pd.concat([users_compensables, users_compensados], ignore_index=True)

# Transformar en formato largo
data_long = pd.melt(data,
                    id_vars=['user_id_order', 'group', 'categoria'],
                    value_vars=['orders_bef', 'order_aft'],
                    var_name='period',
                    value_name='num_orders')

# Cambiar 'period' a 0 (antes) y 1 (después)
data_long['period'] = data_long['period'].apply(lambda x: 0 if x == 'orders_bef' else 1)


<ipython-input-20-0c66bd800f58>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_compensados['group'] = 'treatment'


In [21]:
data_long

,user_id_order,group,categoria,period,num_orders
0,52186541,control,Potential Loyalists,0,1.000000
1,70597167,control,Potential Loyalists,0,2.000000
2,39763782,control,Hibernating,0,4.052632
3,45471287,control,Potential Loyalists,0,2.000000
4,37226902,control,Champion,0,2.529412
...,...,...,...,...,...
48633,36372008,treatment,Champion,1,41.000000
48634,47169380,treatment,Champion,1,1.000000
48635,47169380,treatment,Champion,1,1.000000
48636,47169380,treatment,Champion,1,1.000000


In [22]:
# Champion
import statsmodels.formula.api as smf
model_champion = smf.ols('num_orders ~ group * period', data=data_long[data_long['categoria'] == 'Champion']).fit()
print(model_champion.summary())

                            OLS Regression Results                            
Dep. Variable:             num_orders   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     103.2
Date:                Fri, 08 Nov 2024   Prob (F-statistic):           1.73e-66
Time:                        00:58:41   Log-Likelihood:            -1.1212e+05
No. Observations:               30625   AIC:                         2.242e+05
Df Residuals:                   30621   BIC:                         2.243e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

In [23]:
# Loyal Customers
import statsmodels.formula.api as smf
model_Loyal_Customers = smf.ols('num_orders ~ group * period', data=data_long[data_long['categoria'] == 'Loyal Customers']).fit()
print(model_Loyal_Customers.summary())

                            OLS Regression Results                            
Dep. Variable:             num_orders   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.027
Method:                 Least Squares   F-statistic:                     126.3
Date:                Fri, 08 Nov 2024   Prob (F-statistic):           1.12e-80
Time:                        00:58:53   Log-Likelihood:                -39460.
No. Observations:               13498   AIC:                         7.893e+04
Df Residuals:                   13494   BIC:                         7.896e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

In [24]:
# Potential Loyalists
import statsmodels.formula.api as smf
model_Potential_Loyalists = smf.ols('num_orders ~ group * period', data=data_long[data_long['categoria'] == 'Potential Loyalists']).fit()
print(model_Potential_Loyalists.summary())

                            OLS Regression Results                            
Dep. Variable:             num_orders   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.765
Date:                Fri, 08 Nov 2024   Prob (F-statistic):              0.152
Time:                        00:58:57   Log-Likelihood:                -5292.9
No. Observations:                2417   AIC:                         1.059e+04
Df Residuals:                    2413   BIC:                         1.062e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

In [25]:
# New Customer
import statsmodels.formula.api as smf
model_New_Customer = smf.ols('num_orders ~ group * period', data=data_long[data_long['categoria'] == 'New Customer']).fit()
print(model_New_Customer.summary())

                            OLS Regression Results                            
Dep. Variable:             num_orders   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.8799
Date:                Fri, 08 Nov 2024   Prob (F-statistic):              0.453
Time:                        00:59:01   Log-Likelihood:                -264.49
No. Observations:                 169   AIC:                             537.0
Df Residuals:                     165   BIC:                             549.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

In [26]:
# Hibernating
import statsmodels.formula.api as smf
model_Hibernating = smf.ols('num_orders ~ group * period', data=data_long[data_long['categoria'] == 'Hibernating']).fit()
print(model_Hibernating.summary())

                            OLS Regression Results                            
Dep. Variable:             num_orders   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     2.539
Date:                Fri, 08 Nov 2024   Prob (F-statistic):             0.0572
Time:                        00:59:03   Log-Likelihood:                -370.22
No. Observations:                 246   AIC:                             748.4
Df Residuals:                     242   BIC:                             762.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

In [27]:
# Can't Lose Them
import statsmodels.formula.api as smf
model_Cant_Lose_Them = smf.ols('num_orders ~ group * period', data=data_long[data_long['categoria'] == "Can't Lose Them"]).fit()
print(model_Cant_Lose_Them.summary())

                            OLS Regression Results                            
Dep. Variable:             num_orders   R-squared:                       0.149
Model:                            OLS   Adj. R-squared:                  0.114
Method:                 Least Squares   F-statistic:                     4.208
Date:                Fri, 08 Nov 2024   Prob (F-statistic):            0.00844
Time:                        00:59:07   Log-Likelihood:                -204.27
No. Observations:                  76   AIC:                             416.5
Df Residuals:                      72   BIC:                             425.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

In [28]:
# At Risk
import statsmodels.formula.api as smf
model_At_Risk = smf.ols('num_orders ~ group * period', data=data_long[data_long['categoria'] == "At Risk"]).fit()
print(model_At_Risk.summary())

                            OLS Regression Results                            
Dep. Variable:             num_orders   R-squared:                       0.086
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic:                     11.94
Date:                Fri, 08 Nov 2024   Prob (F-statistic):           1.74e-07
Time:                        00:59:10   Log-Likelihood:                -720.98
No. Observations:                 384   AIC:                             1450.
Df Residuals:                     380   BIC:                             1466.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

In [29]:
# About to Sleep
import statsmodels.formula.api as smf
model_About_Sleep = smf.ols('num_orders ~ group * period', data=data_long[data_long['categoria'] == "About to Sleep"]).fit()
print(model_About_Sleep.summary())

                            OLS Regression Results                            
Dep. Variable:             num_orders   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     9.155
Date:                Fri, 08 Nov 2024   Prob (F-statistic):           5.68e-06
Time:                        00:59:14   Log-Likelihood:                -1948.9
No. Observations:                 910   AIC:                             3906.
Df Residuals:                     906   BIC:                             3925.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

In [30]:
model_champion
model_Loyal_Customers
model_About_Sleep
model_At_Risk
model_Cant_Lose_Them
model_Hibernating
model_Potential_Loyalists
model_New_Customer

In [31]:
import pandas as pd

# Crear un diccionario con cada modelo y su respectiva categoría
modelos = {
    "Champion": model_champion,
    "Loyal Customers": model_Loyal_Customers,
    "About to Sleep": model_About_Sleep,
    "At Risk": model_At_Risk,
    "Can't Lose Them": model_Cant_Lose_Them,
    "Hibernating": model_Hibernating,
    "Potential Loyalists": model_Potential_Loyalists,
    "New Customer": model_New_Customer
}

# Lista para almacenar los resultados
resultados = []

# Extraer el coeficiente y el valor p de cada modelo
for categoria, modelo in modelos.items():
    coef = modelo.params['group[T.treatment]:period']
    p_value = modelo.pvalues['group[T.treatment]:period']
    significativo = 'sí' if p_value < 0.05 else 'no'

    resultados.append({
        "categoria": categoria,
        "Incremental de Ordenes": coef,
        "Estd. Sig.": significativo
    })

# Convertir la lista de resultados en un DataFrame
resumen_df = pd.DataFrame(resultados)

# Mostrar el resumen
print(resumen_df)


             categoria  Incremental de Ordenes Estd. Sig.
0             Champion                1.471612         sí
1      Loyal Customers                0.088607         no
2       About to Sleep                0.229698         no
3              At Risk                0.286915         no
4      Can't Lose Them                1.408458         no
5          Hibernating                0.462599         no
6  Potential Loyalists                0.311825         no
7         New Customer                0.126396         no


In [32]:
# Realizar un left join de resumen_df con resumen_compensables
resumen_merge = resumen_df.merge(resumen_compensables, on='categoria', how='left')

# Realizar un left join del resultado anterior con resumen_compensados
resumen_merge = resumen_merge.merge(resumen_compensados, on='categoria', how='left')

# Mostrar el resultado
print(resumen_merge.head())


         categoria  Incremental de Ordenes Estd. Sig.  users_control  \
0         Champion                1.471612         sí           2195   
1  Loyal Customers                0.088607         no           1283   
2   About to Sleep                0.229698         no            164   
3          At Risk                0.286915         no             90   
4  Can't Lose Them                1.408458         no             14   

   orders_control  freq_antes_control  ticket_antes_control  \
0            2454            8.348234             11.652561   
1            1365            4.970413             11.971607   
2             173            2.439380             10.466871   
3              97            2.854901             11.960070   
4              14            6.095851             15.567411   

   freq_despues_control  ticket_despues_control  gpo_despues_control  ...  \
0              8.087205               11.888607             1.052983  ...   
1              3.515751           

In [33]:
resumen_merge.info(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   categoria                   8 non-null      object 
 1   Incremental de Ordenes      8 non-null      float64
 2   Estd. Sig.                  8 non-null      object 
 3   users_control               8 non-null      int64  
 4   orders_control              8 non-null      int64  
 5   freq_antes_control          8 non-null      float64
 6   ticket_antes_control        8 non-null      float64
 7   freq_despues_control        8 non-null      float64
 8   ticket_despues_control      8 non-null      float64
 9   gpo_despues_control         8 non-null      float64
 10  gpo_antes_control           8 non-null      float64
 11  var freq control            8 non-null      float64
 12  var ticket control          8 non-null      float64
 13  users_tratamiento           8 non-null 

In [34]:
resumen_merge['GPO Average'] = resumen_merge['gpo_despues_tratamiento']
resumen_merge['Monto compensado'] = resumen_merge['monto_comp']
resumen_merge['ROI'] = (resumen_merge['gpo_despues_tratamiento']*resumen_merge['Incremental de Ordenes'])/resumen_merge['monto_comp']
resumen_merge['Freq Before Comp'] = resumen_merge['freq_antes_tratamiento']
resumen_merge['Freq After Comp'] = resumen_merge['freq_despues_tratamiento']

In [35]:
!pip install tabulate

In [36]:
from tabulate import tabulate

# Seleccionar las columnas correctamente
columns_to_display = ['categoria', 'users_control', 'users_tratamiento', 'Incremental de Ordenes', 'Estd. Sig.', 'GPO Average', 'Monto compensado', 'ROI']

# Redondear y formatear valores numéricos, formatear ROI como porcentaje solo si es un número
resumen_merge['ROI'] = resumen_merge['ROI'].apply(lambda x: f"{x * 100:.2f}%" if isinstance(x, (int, float)) else x)

# Mostrar la tabla con tabulate
print(tabulate(resumen_merge[columns_to_display], headers='keys', tablefmt='grid'))


+----+---------------------+-----------------+---------------------+--------------------------+--------------+---------------+--------------------+--------+
|    | categoria           |   users_control |   users_tratamiento |   Incremental de Ordenes | Estd. Sig.   |   GPO Average |   Monto compensado | ROI    |
+====+=====================+=================+=====================+==========================+==============+===============+====================+========+
|  0 | Champion            |            2195 |                8720 |                1.47161   | sí           |      1.19458  |            1.82709 | 96.22% |
+----+---------------------+-----------------+---------------------+--------------------------+--------------+---------------+--------------------+--------+
|  1 | Loyal Customers     |            1283 |                4447 |                0.0886073 | no           |      1.29675  |            2.06557 | 5.56%  |
+----+---------------------+-----------------+------------

**End**